<a href="https://colab.research.google.com/github/selfproclaimedgenius1706/Alzheimer-s-Disease-Detection/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!python3 -m venv venv
!source venv/bin/activate
!pip install tensorflow transformers

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

In [ ]:
df = pd.read_csv('dataToPadding.csv')

In [ ]:
def construct_encodings(data, tokenizer, max_len, truncation=True, padding=True):
    encodings = tokenizer(data, max_length=max_len, truncation=truncation, padding=padding)

    # Warning khi có 1 thằng nào dài hơn max length
    for idx, input_ids in enumerate(encodings['input_ids']):
        if len(input_ids) > max_len:
            print(f"Warning: Sequence at index {idx} is longer than {max_len} tokens.")

    return encodings


In [ ]:
!pip install transformers
from transformers import BertTokenizer

x = df['PreprocessedContent'].tolist()
y = df['Label'].tolist()

# Use the BERT tokenizer
tkzr = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
def construct_tfdataset(encodings, y=None):
    if y:
        return tf.data.Dataset.from_tensor_slices((dict(encodings),y))
    else:
        # this case is used when making predictions on unseen samples after training
        return tf.data.Dataset.from_tensor_slices(dict(encodings))

# Define a function to construct encodings
def construct_encodings(texts, tokenizer, max_len):
    return tokenizer(texts, truncation=True, padding='max_length', max_length=max_len, return_tensors='tf')

encodings = construct_encodings(x, tokenizer=tkzr, max_len=20)

tfdataset = construct_tfdataset(encodings, y)

In [ ]:
#In thử ra xem cho zui
print(encodings['input_ids'][0])
print(encodings['attention_mask'][0])

tf.Tensor(
[  101  2092  2045  2015  1037  2388  3061  2045  7910  7910 12699  1996
 10447  1998  7910  1996  3645  2330  1998   102], shape=(20,), dtype=int32)
tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(20,), dtype=int32)


In [ ]:
#70% for train, 30% for test
TEST_SPLIT = 0.3
BATCH_SIZE = 2
train_size = int(len(x) * (1-TEST_SPLIT))

tfdataset = tfdataset.shuffle(len(x))
tfdataset_train = tfdataset.take(train_size)
tfdataset_test = tfdataset.skip(train_size)

tfdataset_train = tfdataset_train.batch(BATCH_SIZE)
tfdataset_test = tfdataset_test.batch(BATCH_SIZE)

In [ ]:
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras import activations, optimizers, losses


In [ ]:
N_EPOCHS = 10
MODEL_NAME = 'bert-base-uncased'  # Specify the BERT model
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)  # Use BERT's recommended learning rate
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
model.fit(tfdataset_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
38/38 [==============================] - 118s 2s/step - loss: 0.7131 - accuracy: 0.4533
Epoch 2/10
38/38 [==============================] - 84s 2s/step - loss: 0.6526 - accuracy: 0.6933
Epoch 3/10
38/38 [==============================] - 79s 2s/step - loss: 0.5294 - accuracy: 0.7733
Epoch 4/10
38/38 [==============================] - 83s 2s/step - loss: 0.4662 - accuracy: 0.8267
Epoch 5/10
38/38 [==============================] - 79s 2s/step - loss: 0.3566 - accuracy: 0.8667
Epoch 6/10
38/38 [==============================] - 77s 2s/step - loss: 0.2215 - accuracy: 0.9467
Epoch 7/10
38/38 [==============================] - 77s 2s/step - loss: 0.1477 - accuracy: 0.9600
Epoch 8/10
38/38 [==============================] - 75s 2s/step - loss: 0.0835 - accuracy: 0.9867
Epoch 9/10
38/38 [==============================] - 80s 2s/step - loss: 0.0832 - accuracy: 0.9867
Epoch 10/10
38/38 [==============================] - 83s 2s/step - loss: 0.0302 - accuracy: 1.0000


# TEST ON NEW DATASET

In [ ]:
#Check accuracy
benchmarks = model.evaluate(tfdataset_test, return_dict=True, batch_size=BATCH_SIZE)
accuracy = benchmarks['accuracy']
print(f'The accuracy of the model is {round(accuracy*100)}%')

17/17 [==============================] - 10s 278ms/step - loss: 0.0167 - accuracy: 1.0000
The accuracy of the model is 100%


In [ ]:
import torch
testDF = pd.read_csv('preprocessed_data2020.csv')
test_text = testDF['Preprocessed_Content']
test_label = testDF['label']

# Print the shape of the extracted arrays
print("test_text shape:", test_text.shape)
print("test_label shape:", test_label.shape)

tokens_test = tkzr.batch_encode_plus(
    test_text.tolist(),
    max_length = 40,
    pad_to_max_length=True,
    truncation=True
)

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_label.tolist())

test_text shape: (48,)
test_label shape: (48,)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from sklearn.metrics import classification_report
import tensorflow as tf

# Convert PyTorch tensors to TensorFlow tensors
test_seq_tf = tf.convert_to_tensor(test_seq.numpy())
test_mask_tf = tf.convert_to_tensor(test_mask.numpy())

# Use tf.config.list_physical_devices to check for GPU availability
gpu_available = tf.config.list_physical_devices('GPU')

# Use GPU if available, otherwise use CPU
device = "/GPU:0" if gpu_available else "/CPU:0"

with tf.device(device):
    preds = model([test_seq_tf, test_mask_tf])

# Access the logits and convert them to a NumPy array
logits = preds.logits.numpy()
preds = np.argmax(logits, axis=1)
print(classification_report(test_y, preds))


              precision    recall  f1-score   support

           0       0.76      0.67      0.71        24
           1       0.70      0.79      0.75        24

    accuracy                           0.73        48
   macro avg       0.73      0.73      0.73        48
weighted avg       0.73      0.73      0.73        48

